# Data Input

In [1]:
import numpy as np
import pandas as pd 
data= pd.read_excel('C:/Users/shufe/Dropbox/ds new/pwchkt.xlsx')
data.isnull().any(axis=1) # 判定每行会不会有空值的记录。（某条记录但凡在一个变量上有空值，就认为此记录
data=data.dropna(how='any')
data=data[['age','creditamount','guarantors','housing','foreigner','creditrating']]
data.loc[(data.creditrating == 2),'creditrating']=0
data_features=list(data.columns) 
y='creditrating'
data_features.remove(y)
# we also need to create a dictionary indicating whether the feature is discrete or not
discrete={'age':'no','creditamount':'no','guarantors':'yes','housing':'yes','foreigner':'yes','creditrating':'yes'}
data_features

['age', 'creditamount', 'guarantors', 'housing', 'foreigner']

# Function:BestFeature
Defind a function 'BestFeature', which gives the best feature for classfying given the current dataset and feature set.

In [122]:
n_seq=8
def BestFeature(Dataset, unvisited_feature):
    # want to return the best feature and the resulting data split
    # but before this , you can already calculate the the entropy of the current data
    orginal_entropy=entropy_cal(Dataset[y])
    threshold=[]
    entropy_gain=[]
    #print(unvisited_feature)
    for feature in unvisited_feature:
        if discrete[feature]=='yes': # this is discrete 
            # feature is name
            # feature level is number?
            # next, for each level in the feature_level, get
            entropy_gain.append(SplitDataSet(Dataset,feature, 0.5)[0]-orginal_entropy)
            threshold.append(0.5)
        else: # the feature is continuous 
            # we first generate a threshold sequence 
            thres_seq=[   min(Dataset[feature])+i*(max(Dataset[feature])-min(Dataset[feature]))/(n_seq-1) 
                      for i in range(1,n_seq-1)   ]
            entropy_seq= [ SplitDataSet(Dataset,feature, x)[0] for x in thres_seq]
            entropy_gain.append(max(entropy_seq)-orginal_entropy)
            threshold.append(thres_seq[entropy_seq.index(max(entropy_seq))])
        
    # we complete the loop with entropy_gain series
    # we now return the best feature and the corresponding threshold 
    # for discrete, the corresponding threshold is always -1 
    best_feature=unvisited_feature[entropy_gain.index(max(entropy_gain))]
    best_threshold=threshold[entropy_gain.index(max(entropy_gain))]
    # we also return the splitted dat
    return best_feature,best_threshold
    # best_feature is a string. best_threshold is a value. 
  

# Function: SplitDataSet
Split the current dataset by the given feature at given threshold value.

In [79]:
def SplitDataSet(Dataset,feature, value):
# we only consider the binary classification case
# given the dataset, feature and value, this function returns splitted data and the resulting entropy!
    Data_left=Dataset[Dataset[feature]<value]
    Data_right=Dataset[Dataset[feature]>=value]
# we want to calculate the entropy of the data_left, data_right
    Total_entropy=entropy_cal(Data_left[y])*len(Data_left.index)/len(Dataset.index)+entropy_cal(Data_right[y])*len(Data_right.index)/len(Dataset.index)
    return Total_entropy,Data_left, Data_right 


# Function: Entropy_Cal
Calculate entropy of target variable for given dataset.

In [102]:
import math
def entropy_cal(y_data):
    p=sum(y_data==1)/len(y_data)
    if (p==1 or p==0):
        return(0)
    else:
        return(-p*math.log(p)-(1-p)*math.log(1-p))

# Main Program: createTree
This is recursive function that creates a tree.

In [121]:
def createTree(Dataset, layer,features):
    # we first need to check whether the layer has already reached the max
    if (layer==maxlayer  ):
        # you have already reached the deepest. all you have to do is to return the class that has the most sample
        n_positive=sum(Dataset[y]==1)
        n_negative=sum(Dataset[y]==0)
        Tree=1*(n_positive>n_negative)+0*(n_positive<=n_negative)
        
    elif (len(set(Dataset[y]))==1):
        Tree=Dataset[0] # since every one in the dataset has the same label.
    else:
        best_feature,best_threshold=BestFeature(Dataset, features)
        Tree = {}  # means that you want to generate a new son tree
        # the new son tree should contain the following features:
        Tree['spInd'] = best_feature # the feature you use to split the data
        featuresNext=features
        featuresNext.remove(best_feature)  # this feature would no longer be used in the next trees
        Tree['spVal'] = best_threshold # the threshold value 
        data_left, data_right = SplitDataSet(Dataset,best_feature, best_threshold)[1],SplitDataSet(Dataset,best_feature, best_threshold)[2]
        Tree['left'] = createTree(data_left,layer+1,featuresNext)
        Tree['right'] = createTree(data_right,layer+1,featuresNext)
    return Tree

        

# Test

In [133]:
maxlayer=3
createTree(data,1,data_features)

{'spInd': 'guarantors',
 'spVal': 0.5,
 'left': {'spInd': 'age', 'spVal': 51.0, 'left': 1, 'right': 1},
 'right': {'spInd': 'creditamount',
  'spVal': 11450.142857142857,
  'left': 1,
  'right': 1}}